In [33]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from aac_crud import AnimalShelterCRUD

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU2024"
host = "nv-desktop-services.apporto.com"
port = 31216
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
db = AnimalShelterCRUD(username, password, host, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Austin Scarinza'))),
    html.Hr(),
    html.Div(className='buttonRow', 
            style={'display' : 'flex'},
                children=[
                    html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                    html.Button(id='submit-button-two', n_clicks=0, children='Mountain or Wilderness Rescue'),
                    html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                    html.Button(id='submit-button-four', n_clicks=0, children='Reset'),
                ]),
              
     dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=10,
                        ),

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(
        Output('datatable-id', 'data'),
        [Input('submit-button-one', 'n_clicks'),
         Input('submit-button-two', 'n_clicks'),
         Input('submit-button-three', 'n_clicks'),
         Input('submit-button-four', 'n_clicks')])

def on_click(n_clicks_one, n_clicks_two, n_clicks_three, n_clicks_four):
    
    # start case
    query = {}
    
    # Keep Type Error for occuring
    n_clicks_one = n_clicks_one or 0
    n_clicks_two = n_clicks_two or 0
    n_clicks_three = n_clicks_three or 0
    n_clicks_four = n_clicks_four or 0
    
    if n_clicks_one > 0:
        query = {'animal_type': 'Dog', 'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 
                 'sex_upon_outcome': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gt': 26, '$lt': 156}}
    elif n_clicks_two > 0:
        query = {'animal_type': 'Dog', 'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Huskey', 'Rottweiler']}, 
                 'sex_upon_outcome': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gt': 26, '$lt': 156}}
    elif n_clicks_three > 0:
        query = {'animal_type': 'Dog', 'breed': {'$in': ['Doberman Pinsher', 'German Shepard', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}, 
                 'sex_upon_outcome': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gt': 20, '$lt': 300}}
    elif n_clicks_four > 0:
        query = {}
        
    
    # Get data using query
    df = pd.DataFrame.from_records(db.read(query))
    
    # Cleanup Mongo _id field
    df.drop(columns=['_id'],inplace=True)
    
    return df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Preferred Animals')
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '500px', 'height': '250px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:31200/
